<a href="https://colab.research.google.com/github/OlanreleOladeji/Demand-forecast--XGBoost/blob/main/Products_quantity_forecasting_model_(XGB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data upload and preprocessing

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [7]:
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
df = pd.read_csv('1pharma-data.csv')

In [ ]:
df.head()

Check dataset structure

In [ ]:
df.info()

Data Exploration and Cleaning

Print the column headings

In [ ]:
print(df.columns.tolist())


Dropping columns

In [ ]:
df= df.drop(['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Price','Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Distributor', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Class', 'Unnamed: 18'], axis=1)

In [ ]:
df.head()

Convert the month column to integer

In [ ]:
#month_to_int  = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

In [ ]:
#df['Month'] = df['Month'].map(month_to_int)

In [ ]:
#df.head(10)

Sum the number of items in each month per year

In [ ]:
monthly_sales = df.groupby(['Year', 'Month', 'Product_ID'])['Sales'].sum().reset_index()

In [ ]:
Monthly_Revenue= df.groupby(['Year', 'Month', 'Product_ID'])['Revenue'].sum().reset_index()

In [ ]:
merged_df = pd.merge(monthly_sales, Monthly_Revenue, on=['Year', 'Month', 'Product_ID'])

In [ ]:
merged_df.head()

In [ ]:
df= merged_df

In [ ]:
df

In [ ]:
#print(df['Product_ID'].unique())

Feature Engineering

Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split
X = df[['Revenue']]
y = df[['Sales']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X

In [ ]:
y

Dataset Scaling using MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

Scaling the train_X and test_X :

In [ ]:
scaler_X = MinMaxScaler()

In [ ]:
scaler_X.fit(X)

In [ ]:
train_X_Scaled = scaler_X.transform(train_X)

In [ ]:
train_X_Scaled = pd.DataFrame(train_X_Scaled, columns=['Revenue'])

In [ ]:
train_X_Scaled.head(10)

In [ ]:
test_X_Scaled = scaler_X.transform(test_X)

In [ ]:
test_X_Scaled = pd.DataFrame(test_X_Scaled, columns=['Revenue'])

In [ ]:
test_X_Scaled.head(10)

Scaling the train_y and test_y :

In [ ]:
scaler_y = MinMaxScaler()

In [ ]:
scaler_y.fit(y)

MinMaxScaler()

In [ ]:
train_y_Scaled = scaler_y.transform(train_y)

In [ ]:
train_y_Scaled = pd.DataFrame(train_y_Scaled, columns=['Sales'])

In [ ]:
train_y_Scaled.head(10)

,Sales
0,0.061046
1,0.062750
2,0.070812
3,0.061553
4,0.059503
5,0.056642
6,0.076824
7,0.055990
8,0.053812
9,0.061569


In [ ]:
test_y_Scaled = scaler_y.transform(test_y)

In [ ]:
test_y_Scaled = pd.DataFrame(test_y_Scaled, columns=['Sales'])

In [ ]:
test_y_Scaled.head(10)

,Sales
0,0.053989
1,0.058418
2,0.056537
3,0.093880
4,0.069936
5,0.057445
6,0.056199
7,0.091831
8,0.055291
9,0.070490


XGBoost Algorithm

In [2]:
from xgboost import XGBRegressor

In [3]:
model = XGBRegressor()

In [4]:
model.fit(train_X_Scaled, train_y_Scaled)

NameError: name 'train_X_Scaled' is not defined

In [1]:
print(model)

NameError: name 'model' is not defined

In [ ]:
model.predict(test_X)

array([0.32182753, 0.32182753, 0.32182753, ..., 0.32182753, 0.32182753,
       0.32182753], dtype=float32)

In [ ]:
y_predict = model.predict(test_X_Scaled)

Evaluate model with performance metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error

In [ ]:
mean_squared_error(test_y_Scaled, y_predict)

0.00038125834544189274

In [ ]:
mean_absolute_error(test_y_Scaled, y_predict)

0.008018013089895248

In [ ]:
 root_mean_squared_error(test_y_Scaled, y_predict)

0.019525837153196335

In [ ]:
#pip install m5-wrmsse

In [ ]:
#from m5_wrmsse import wrmsse

In [ ]:
#y_predict = np.ones((30490,28))

In [ ]:
#wrmsse_score = wrmsse(y_predict)

In [ ]:
#print(wrmsse_score)

Model tunning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "learning_rate": [0.01, 0.1],
    "max_depth": [3, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
   }

In [ ]:
grid_search = GridSearchCV(estimator= XGBRegressor(), param_grid= param_grid, cv=5, verbose=1)

In [ ]:
grid_search.fit(train_X_Scaled, train_y_Scaled)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 7],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000],
                         'subsample': [0.8, 1.0]},
             verbose=1)

In [ ]:
grid_search.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100,
 'subsample': 0.8}

In [ ]:
grid_search.best_estimator_.predict(test_X_Scaled)

array([0.06137752, 0.06510728, 0.06286702, ..., 0.06199182, 0.06261616,
       0.06199182], dtype=float32)

In [ ]:
y_predict_tuned = grid_search.best_estimator_.predict(test_X_Scaled)

In [ ]:
mean_squared_error(test_y_Scaled, y_predict_tuned)

0.00038102251710370183

In [ ]:
mean_absolute_error(test_y_Scaled, y_predict_tuned)

0.007939578965306282

In [ ]:
 root_mean_squared_error(test_y_Scaled, y_predict_tuned)

0.019519798457622528

In [ ]:
#from m5_wrmsse import wrmsse

In [ ]:
#y_pred_tuned = np.ones((30490,28))

In [ ]:
#wrmsse_score = wrmsse(y_pred_tuned)
#print(wrmsse_score)

In [ ]:
#month_to_int = {
 #   'January': 1,'February': 2,'March': 3,
  #  'April': 4,'May': 5,'June': 6,'July': 7,
   # 'August': 8,'September': 9,'October': 10,
    #'November': 11,'December': 12
#}

#df['Month'] = df['Month'].map(month_to_int)

#df.head()